In [23]:
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('~')
from analysts_tools.growth import *
    #Procurement tools
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

In [24]:
city = 'CMX'
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')

In [25]:
def filter_customers(city, recency_day=20, kam=100, graduate=100, msgt=('ALL'), sgt=('ALL')):
    """
    This function filters the customers that have completed the specified criteria:
    city: The city that the customer has been registered -> i.e 'BOG'
    recency_day: The limit of days that the customer has completed an order -> 30 means 30 days or less
    kam: If the customer is a KAM or No -> {100: ALL, 1: True, 0: False}
    graduate: If the customer is GRADUATED or No -> {100: ALL, 1: True, 0: False}
    msgt: The microsegment of the customer, this is different based on the city
        - If you want to bring all should put as a tuple ('ALL')
        - If you want to bring one msgt then i.e ('Fonda')
        - If you want to bring more than one msgt then i.e ('Fonda','Corrientazo')
    sgt: The segment of the customer
        - If you want to bring all should put as a tuple ('ALL')
        - If you want to bring one msgt then i.e ('Restaurante')
        - If you want to bring more than one msgt then i.e ('Comercio','Restaurante')
    """
    import pandas as pd
    from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

    if isinstance(msgt, str):
        msgt = (msgt,'a')  # Convert the string to a tuple with a single element
    if isinstance(sgt, str):
        sgt = (sgt,'a')  # Convert the string to a tuple with a single element

    query = """
    SELECT
        source_id AS customer_id,
        is_kam,
        birthday,
        last_order_date,
        recency_days,
        registered_city AS region,
        microsegment_name AS microsegment,
        business_type AS segment,
        is_graduated
    FROM dpr_customer.obt_customer

    WHERE recency_days <= {days}
    ----------------------------------
    ----------------------------------
    AND 
    CASE
    WHEN '{city}' = 'ALL' THEN 1=1
    ELSE registered_city = '{city}'
    END
    ----------------------------------
    ----------------------------------
    AND
    CASE
    WHEN {is_kam} = 100 THEN 1=1
    ELSE is_kam = {is_kam}
    END
    ----------------------------------
    ----------------------------------
    AND
    CASE
    WHEN {graduated} = 100 THEN 1=1
    ELSE is_graduated = {graduated}
    END
    ----------------------------------
    ----------------------------------
    AND 
    CASE
    WHEN 'ALL' IN {microsegment} THEN 1=1
    ELSE microsegment_name IN {microsegment}
    END
    ----------------------------------
    ----------------------------------
    AND
    CASE
    WHEN 'ALL' IN {segment} THEN 1=1
    ELSE business_type IN {segment}
    END
    """.format(days=recency_day,city=city,is_kam=kam,graduated=graduate,microsegment=msgt,segment=sgt)

    return run_read_dwd_query(query)

In [26]:
def info_sells(city, sells_days_back_start=30, days_end=-1, customer=None):    
    """
    This returns a df with the information of the whole sells order by order based on the restrictions
    city: The city that the customer has been registered -> i.e 'BOG'
    sells_days_back_start: Since when we want to see the orders information
    days_end: Until when we want to see the orders information
    """    
    import pandas as pd
    from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
    import time
    import datetime
    
    sells_date_inicio = (datetime.datetime.today() - datetime.timedelta(days=sells_days_back_start)).strftime('%Y-%m-%d') #1 months
    sells_date_fin = (datetime.datetime.today() - datetime.timedelta(days=days_end)).strftime('%Y-%m-%d') #1 months
    
    tasas = {'BOG': 3776,
            'BAQ': 3776,
            'MDE': 3776,
            'CMX': 19.65,
            'GDL': 19.65,
            'PBC': 19.65,
            'SPO': 4.75,
            'BHZ': 4.75,
            'CWB': 4.75,
            'VCP': 4.75}

    if not isinstance(customer, list):
        raise TypeError("Customer parameter must be a list.")
    elif customer is None:    
        query = """
        SELECT 
            s.identifier_value AS region_code,
            DATE(fs.order_submitted_date) AS submit_date,
            fs.order_close_date AS close_date,
            fs.order_id,
            dc.source_id AS customer_id,
            cat.parent_description as category,
            cat.description AS subcategory,
            dp.card_id,
            dp.source_id AS sku_id,
            dp.card_description AS product_name,
            fs.product_quantity_x_step_unit AS cant,
            fs.gmv_pxq_local/{tasa_cambio} AS gmv_usd,
            COALESCE(fsd.product_discount/{tasa_cambio},0) AS discount_applied,
            fsd.adjustment_reason,
            COALESCE(bo.source_id,NULL) AS offer_id
            
        FROM dpr_sales.fact_sales                   fs
        INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
        INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
        INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
        INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
        LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
        LEFT JOIN dpr_sales.dim_offer               bo  ON bo.offer_id = fsd.dim_offer     

        WHERE 
            fs.gmv_enabled = TRUE
            AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
            AND fs.fb_order_status_id  IN (1,6,7,8)
            AND fs.is_deleted = FALSE
            AND fs.dim_status = 1
            AND dp.is_slot = 'false'
            --AND fsd.is_enabled = true
            AND s.identifier_value = '{ciudad}'
            AND DATE(fs.order_submitted_date) between '{Fecha_start}' AND '{Fecha_end}'
    """.format(tasa_cambio = tasas[city], ciudad = city, Fecha_start = sells_date_inicio, Fecha_end = sells_date_fin)
    else:
        query = """
        SELECT 
            s.identifier_value AS region_code,
            DATE(fs.order_submitted_date) AS submit_date,
            fs.order_close_date AS close_date,
            fs.order_id,
            dc.source_id AS customer_id,
            cat.parent_description as category,
            cat.description AS subcategory,
            dp.card_id,
            dp.source_id AS sku_id,
            dp.card_description AS product_name,
            fs.product_quantity_x_step_unit AS cant,
            fs.gmv_pxq_local/{tasa_cambio} AS gmv_usd,
            COALESCE(fsd.product_discount/{tasa_cambio},0) AS discount_applied,
            fsd.adjustment_reason,
            COALESCE(bo.source_id,NULL) AS offer_id
            
        FROM dpr_sales.fact_sales                   fs
        INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
        INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
        INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
        INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
        LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
        LEFT JOIN dpr_sales.dim_offer               bo  ON bo.offer_id = fsd.dim_offer     

        WHERE 
            fs.gmv_enabled = TRUE
            AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
            AND fs.fb_order_status_id  IN (1,6,7,8)
            AND fs.is_deleted = FALSE
            AND fs.dim_status = 1
            AND dp.is_slot = 'false'
            --AND fsd.is_enabled = true
            AND s.identifier_value = '{ciudad}'
            AND DATE(fs.order_submitted_date) between '{Fecha_start}' AND '{Fecha_end}'
            AND dc.source_id IN {clientes}
    """.format(tasa_cambio = tasas[city], ciudad = city, Fecha_start = sells_date_inicio, Fecha_end = sells_date_fin,clientes=tuple(set(customer)))   
    return run_read_dwd_query(query)

In [27]:
df_customers = filter_customers(city,40,0,100).customer_id.unique()

In [28]:
df_customers

array([348190048, 348228096, 348207237, ..., 264803653,  30348487,
         -260407])

In [29]:
df_sells = info_sells(city,customer=df_customers.tolist())

In [30]:
df_sells.loc[~df_sells.customer_id.isin(df_sells.loc[(df_sells.category == 'Carne, pollo & pescados')].customer_id.unique())].customer_id.unique()

array([293750489,  66190626,  59204390, ...,  39815445, 233214447,
        61060250])

In [31]:
customers = df_sells.loc[~df_sells.customer_id.isin(df_sells.loc[(df_sells.category == 'Carne, pollo & pescados')].customer_id.unique())].customer_id.unique()

In [32]:
len(customers)

8718

In [33]:
df_sells

,region_code,submit_date,close_date,order_id,customer_id,category,subcategory,card_id,sku_id,product_name,cant,gmv_usd,discount_applied,adjustment_reason,offer_id
0,CMX,2023-06-07,2023-06-08,21997423,133258425,"Carne, pollo & pescados",Res,229372,328683,Suadero Picado de Res,2.0000,11.07379134,1.3292620,DAY_PROTEINS_MAR_CMX_W23,-14507702.0
1,CMX,2023-06-07,2023-06-08,21999961,293750489,Frutas & verduras,Frutas,313503,412662,Limón con semilla 2a,1.0000,0.74605597,0.0371501,Merma Fruver CMX 25 04 23,257525.0
2,CMX,2023-06-07,2023-06-08,22000313,15814737,Lácteos & huevos,Quesos,454131,629083,Queso Imitación Mozzarella 1 Kg,1.0000,5.39440203,0.2157760,ACMKTPLC_TIME_CCP__4%_21-04-2023_CMX_El Campan...,-14503697.0
3,CMX,2023-06-07,2023-06-08,21976493,7401024,Frutas & verduras,Tubérculos,55101,66520,Papa Blanca 2a,2.0000,3.35165394,0.5027989,DAY_FRUVER_MID_MIE_CMX_W23,-14509266.0
4,CMX,2023-06-07,2023-06-08,22002987,66190626,Frutas & verduras,Verduras,403462,560436,Chile Cuaresmeño 2a,6.0000,5.68549618,0.8519083,DAY_FRUVER_TAIL_MIE_CMX_W23,-14509265.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246836,CMX,2023-07-02,2023-07-03,22598267,22016305,Frutas & verduras,Verduras,-201505,-304926,Pimiento,2.0000,2.78880407,0E-7,None,NaN
246837,CMX,2023-07-02,2023-07-03,22552459,-263126,Frutas & verduras,Verduras,-201498,-304923,Verdolagas,3.0000,3.49618320,0E-7,None,NaN
246838,CMX,2023-07-02,2023-07-03,22597996,52895169,Abarrotes & despensa,Sal & sazonadores,-201576,-304536,Pimienta Negra x 100 gr,0.2000,1.24753180,0E-7,None,NaN
246839,CMX,2023-07-02,2023-07-03,22604991,297677996,Frutas & verduras,Verduras,-201464,-303576,Jitomate Cherry,0.5000,1.28244274,0E-7,None,NaN


In [34]:
df_sells.groupby(['customer_id']).agg({'gmv_usd':np.sum}).reset_index()

,customer_id,gmv_usd
0,-269919,249.45547028
1,-269896,349.87175570
2,-269885,118.15445273
3,-269877,68.67633587
4,-269871,80.29078876
...,...,...
11233,348208795,22.56793890
11234,348212929,36.08295163
11235,348216464,178.02290073
11236,348224503,46.03816790


In [35]:
# from sklearn.model_selection import train_test_split 

# def create_groups(customers):
    
#     offer_df_aux = df_sells.groupby(['customer_id']).agg({'gmv_usd':np.sum}).reset_index()
    
#     customer_control, customer_test = train_test_split(customers, test_size = 0.50)
#     alpha=0.01
#     aux=0
#     while abs(1-(offer_df_aux[offer_df_aux.customer_id.isin(customer_control)]['gmv_usd'].mean()/offer_df_aux[offer_df_aux.customer_id.isin(customer_test)]['gmv_usd'].mean())) > alpha:
#         customer_control, customer_test = train_test_split(customers, test_size = 0.50)
        
#         aux+=1
#         if aux == 1000:
#             alpha+=0.01
#             aux=0
        
#     return customer_control.tolist(), customer_test.tolist()

In [36]:
# control,test = create_groups(customers)

global_segment_list = [] #//for procesing
# offer_name = []

# offer_name.append("LEVELS_PROTEINS_1_"+todays_date+"_"+city)
d = {'name':"LEVELS_PROTEINS_1_"+todays_date+"_"+city, 'customersIds': customers.tolist()}
#e = {'name':"LEVELS_PROTEINS_1_"+todays_date+"_"+city+"_CONTROL", 'customersIds': list(control)}
    
global_segment_list.append(d)
#global_segment_list.append(e)

In [37]:
import requests
import json

url = f"https://{city}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [38]:
print(response.text)

Customer segments were created: [187451-LEVELS_PROTEINS_1_2023-07-07_CMX]


In [39]:
# Create a DataFrame
df = pd.DataFrame({'city': [city] * len(customers), 'customer_id': customers})

# Export DataFrame to an Excel file
df.to_excel(f'{city}_PROTEINS_WEEK_1_CUSTOMERS.xlsx', index=False)

In [137]:
# import requests
# import json

# url = f"https://{city}.frubana.com/api/v1/segments/massive/update?api-key=00_growth-team-zzz-qqq_001"

# payload = json.dumps({
#   "customerSegmentId": 180351,
#   "customersIds": customers.tolist()
# })
# headers = {
#   'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=QckILmAqGVidiFYhvXurB0V1V8Tc8NQStAXtm5jInP6GLYpn8MTYs5OniXDFTMvES9Hi67mD/6BrU6R0esega1J//Cj6FW97TF5OLIKtyRhJyouq7pc6J2I1do0T; AWSALBCORS=QckILmAqGVidiFYhvXurB0V1V8Tc8NQStAXtm5jInP6GLYpn8MTYs5OniXDFTMvES9Hi67mD/6BrU6R0esega1J//Cj6FW97TF5OLIKtyRhJyouq7pc6J2I1do0T',
#   'Content-Type': 'application/json'
# }

# response = requests.request("POST", url, headers=headers, data=payload)

# print(response.text)

Error CustomerSegment: 180351 - the customer 310585971 no was found
